In [82]:
# LSTMのサンプル
#  https://www.tensorflow.org/tutorials/sequences/recurrent

# 実行環境の出力
!python --version

Python 3.5.2


In [107]:
# 必要ライブラリのインポート
import collections
import os

import numpy as np
import tensorflow as tf

import time

In [84]:
# データを読み込むためのメソッド群
def read_words(filename):
    with tf.gfile.GFile(filename, "r") as f:
        return f.read().replace("\n", "<eos>").split()

def build_vocab(filename):
    data = read_words(filename)
    
    # counter.items() = [('voc', cnt), ...]
    # key=lambda x: (-x[1], x[0])とすることでcntの降順かつcntが同じときは単語の昇順になる
    # count_pairs = [('the', 50770), ('<unk>', 45020), ('<eos>', 42068), ... ]
    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))
    
    return word_to_id

def file_to_word_ids(filename, word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]

# データの読み込み
data_path = '/tf/notebooks/data/ptb'
train_path = os.path.join(data_path, "ptb.train.txt")
valid_path = os.path.join(data_path, "ptb.valid.txt")
test_path = os.path.join(data_path, "ptb.test.txt")

word_to_id = build_vocab(train_path)
train_data = file_to_word_ids(train_path, word_to_id)
valid_data = file_to_word_ids(valid_path, word_to_id)
test_data = file_to_word_ids(test_path, word_to_id)
vocabulary = len(word_to_id)

In [85]:
# ↑で作成したdataを実際に使うときのためのメソッドを定義
def ptb_producer(raw_data, batch_size, num_steps, name=None):
    """
    raw_dataをtensorに変換して、batch_sizeとnum_stepsからわかる該当のデータを返す
    """
    with tf.name_scope(name, "PTBProducer", [raw_data, batch_size, num_steps]):
        raw_data = tf.convert_to_tensor(raw_data, name="raw_data", dtype=tf.int32)
        
        data_len = tf.size(raw_data)
        batch_len = data_len // batch_size
        data = tf.reshape(raw_data[0 : batch_size * batch_len], [batch_size, batch_len])
        
        epoch_size = (batch_len - 1) // num_steps
        assertion = tf.assert_positive(epoch_size, message="epoch_size == 0, decrease batch_size or num_steps")
        
        with tf.control_dependencies([assertion]):
            epoch_size = tf.identity(epoch_size, name="epoch_size")
            
        i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()
        x = tf.strided_slice(data, [0, i * num_steps], [batch_size, (i + 1) * num_steps])
        x.set_shape([batch_size, num_steps])
        y = tf.strided_slice(data, [0, i * num_steps + 1], [batch_size, (i + 1) * num_steps + 1])
        y.set_shape([batch_size, num_steps])
        
        return x, y

In [86]:
# 学習のパラメータ
class Config(object):
    init_scale = 0.1
    learning_rate = 1.0
    max_grad_norm = 5
    num_layers = 2
    num_steps = 20
    hidden_size = 200
    max_epoch = 4
    max_max_epoch = 13
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 20
    vocab_size = 10000
    rnn_mode = "block"
    
config = Config()
eval_config = Config()
eval_config.batch_size = 1
eval_config.num_steps = 1

In [87]:
# 入力用のクラス
class PTBInput(object):
    def __init__(self, config, data, name=None):
        self.batch_size = batch_size = config.batch_size
        self.num_steps = num_steps = config.num_steps
        self.epoch_size = ((len(data) // batch_size) - 1) // num_steps
        self.input_data, self.targets = ptb_producer(data, batch_size, num_steps, name=name)

In [104]:
# 学習モデルを定義したクラス
class PTBModel(object):
    def __init__(self, is_training, config, input_):
        self._is_training = is_training
        self._input = input_
        self._rnn_params = None
        self._cell = None
        self.batch_size = input_.batch_size
        self.num_steps = input_.num_steps
        size = config.hidden_size
        vocab_size = config.vocab_size
        
        # Networkの定義
        # embedding -> rnn(lstm) -> softmax
        with tf.device("/cpu:0"):
            embedding = tf.get_variable("embedding", [vocab_size, size], dtype=tf.float32)
            inputs = tf.nn.embedding_lookup(embedding, input_.input_data)
            
        if is_training and config.keep_prob < 1:
            inputs = tf.nn.dropout(inputs, config.keep_prob)
        
        output, state = self._build_rnn_graph_lstm(inputs, config, is_training)
        
        softmax_w = tf.get_variable("softmax_w", [size, vocab_size], dtype=tf.float32)
        softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=tf.float32)
        logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
        logits = tf.reshape(logits, [self.batch_size, self.num_steps, vocab_size])
        
        loss = tf.contrib.seq2seq.sequence_loss(
            logits,
            input_.targets,
            tf.ones([self.batch_size, self.num_steps], dtype=tf.float32),
            average_across_timesteps=False,
            average_across_batch=True
        )
        
        self._cost = tf.reduce_sum(loss)
        self._final_state = state
        
        if not is_training:
            return
        
        # training時は重みの更新を行う
        self._lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self._cost, tvars), config.max_grad_norm)
        optimizer = tf.train.GradientDescentOptimizer(self._lr)
        self._train_op = optimizer.apply_gradients(zip(grads, tvars), global_step=tf.train.get_or_create_global_step())
        
        self._new_lr = tf.placeholder(tf.float32, shape=[], name="new_learning_rate")
        self._lr_update = tf.assign(self._lr, self._new_lr)
        
        
    def _build_rnn_graph_lstm(self, inputs, config, is_training):
        def make_cell():
            cell = tf.contrib.rnn.LSTMBlockCell(config.hidden_size, forget_bias=0.0)
            if is_training and config.keep_prob < 1:
                cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=config.keep_prob)
            return cell
        
        cell = tf.contrib.rnn.MultiRNNCell([make_cell() for _ in range(config.num_layers)], state_is_tuple=True)
        
        self._initial_state = cell.zero_state(config.batch_size, tf.float32)
        state = self._initial_state
        
        # tutorialではコメントアウトされている方で実装してみた
        inputs = tf.unstack(inputs, num=self.num_steps, axis=1)
        outputs, state = tf.nn.static_rnn(cell, inputs, initial_state=self._initial_state)
        
        output = tf.reshape(tf.concat(outputs, 1), [-1, config.hidden_size])
        return output, state
    
    def assign_lr(self, session, lr_value):
        session.run(self._lr_update, feed_dict={self._new_lr: lr_value})
    
    def export_ops(self, name):
        def export_state_tuples(state_tuples, name):
            for state_tuple in state_tuples:
                tf.add_to_collection(name, state_tuple.c)
                tf.add_to_collection(name, state_tuple.h)
        
        self._name = name
        ops = {self._with_prefix(self._name, "cost"): self._cost}
        if self._is_training:
            ops.update(lr=self._lr, new_lr=self._new_lr, lr_update=self._lr_update)
        for name, op in ops.items():
            tf.add_to_collection(name, op)
        self._initial_state_name = self._with_prefix(self._name, "initial")
        self._final_state_name = self._with_prefix(self._name, "final")
        export_state_tuples(self._initial_state, self._initial_state_name)
        export_state_tuples(self._final_state, self._final_state_name)
    
    def import_ops(self):
        def import_state_tuples(state_tuples, name, num_replicas):
            restored = []
            for i in range(len(state_tuples) * num_replicas):
                c = tf.get_collection_ref(name)[2 * i + 0]
                h = tf.get_collection_ref(name)[2 * i + 1]
                restored.append(tf.contrib.rnn.LSTMStateTuple(c, h))
            return tuple(restored)
        
        if self._is_training:
            self._train_op = tf.get_collection_ref("train_op")[0]
            self._lr = tf.get_collection_ref("lr")[0]
            self._new_lr = tf.get_collection_ref("new_lr")[0]
            self._lr_update = tf.get_collection_ref("lr_update")[0]
        self._cost = tf.get_collection_ref(self._with_prefix(self._name, "cost"))[0]
        num_replicas = 1
        self._initial_state = import_state_tuples(self._initial_state, self._initial_state_name, num_replicas)
        self._final_state = import_state_tuples(self._final_state, self._final_state_name, num_replicas)
    
    def _with_prefix(self, prefix, name):
            return "/".join((prefix, name))
        
    @property
    def cost(self):
        return self._cost
    
    @property
    def lr(self):
        return self._lr
    
    @property
    def train_op(self):
        return self._train_op
    
    @property
    def initial_state(self):
        return self._initial_state
    
    @property
    def final_state(self):
        return self._final_state
    
    @property
    def input(self):
        return self._input

In [113]:
def run_epoch(session, model, eval_op=None, verbose=False):
    start_time = time.time()
    costs = 0.0
    iters = 0
    state = session.run(model.initial_state)
    fetches = {
        "cost": model.cost,
        "final_state": model.final_state,
    }
    if eval_op is not None:
        fetches["eval_op"] = eval_op
        
    for step in range(model.input.epoch_size):
        feed_dict = {}
        for i, (c, h) in enumerate(model.initial_state):
            feed_dict[c] = state[i].c
            feed_dict[h] = state[i].h
            
        vals = session.run(fetches, feed_dict)
        cost = vals["cost"]
        state = vals["final_state"]

        costs += cost
        iters += model.input.num_steps

        if verbose and step % (model.input.epoch_size // 10) == 10:
            print("%.3f perplexity: %.3f speed: %.0f wps" % (step * 1.0 / model.input.epoch_size, np.exp(costs / iters), iters * model.input.batch_size * max(1, FLAGS.num_gpus) / (time.time() - start_time)))
                
        return np.exp(costs / iters)

In [114]:
# 学習を行う
with tf.Graph().as_default():
    initializer = tf.random_uniform_initializer(-config.init_scale, config.init_scale)
    
    with tf.name_scope("Train"):
        train_input = PTBInput(config=config, data=train_data, name="TrainInput")
        with tf.variable_scope("Model", reuse=None, initializer=initializer):
            m = PTBModel(is_training=True, config=config, input_=train_input)
        tf.summary.scalar("Training_Loss", m.cost)
        tf.summary.scalar("Learning_Rate", m.lr)
        
    with tf.name_scope("Valid"):
        valid_input = PTBInput(config=config, data=valid_data, name="ValidInput")
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            mvalid = PTBModel(is_training=False, config=config, input_=valid_input)
        tf.summary.scalar("Validation_Loss", mvalid.cost)
        
    with tf.name_scope("Test"):
        test_input = PTBInput(config=eval_config, data=test_data, name="TestInput")
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            mtest = PTBModel(is_training=False, config=eval_config, input_=test_input)
            
    models = {"Train": m, "Valid": mvalid, "Test": mtest}
    for name, model in models.items():
        model.export_ops(name)
    metagraph = tf.train.export_meta_graph()
    soft_placement = False
    
with tf.Graph().as_default():
    tf.train.import_meta_graph(metagraph)
    for model in models.values():
        model.import_ops()
    sv = tf.train.Supervisor(logdir=None)
    config_proto = tf.ConfigProto(allow_soft_placement=soft_placement)
    with sv.managed_session(config=config_proto) as session:
        for i in range(config.max_max_epoch):
            lr_decay = config.lr_decay ** max(i + 1 - config.max_epoch, 0.0)
            m.assign_lr(session, config.learning_rate * lr_decay)

            print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
            train_perplexity = run_epoch(session, m, eval_op=m.train_op, verbose=True)
            print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
            valid_perplexity = run_epoch(session, mvalid)
            print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))

        test_perplexity = run_epoch(session, mtest)
        print("Test Perplexity: %.3f" % test_perplexity)

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.
Epoch: 1 Learning rate: 1.000
Epoch: 1 Train Perplexity: 9988.903
Epoch: 1 Valid Perplexity: 7444.862
Epoch: 2 Learning rate: 1.000
Epoch: 2 Train Perplexity: 7374.813
Epoch: 2 Valid Perplexity: 4057.475
Epoch: 3 Learning rate: 1.000
Epoch: 3 Train Perplexity: 4083.982
Epoch: 3 Valid Perplexity: 325166.336
Epoch: 4 Learning rate: 1.000
Epoch: 4 Train Perplexity: 277633.834
Epoch: 4 Valid Perplexity: 3322.948
Epoch: 5 Learning rate: 0.500
Epoch: 5 Train Perplexity: 4257.768
Epoch: 5 Valid Perplexity: 3667.908
Epoch: 6 Learning rate: 0.250
Epoch: 6 Train Perplexity: 3635.160
Epoch: 6 Valid Perplexity: 2447.867
Epoch: 7 Learning rate: 0.125
Epoch: 7 Train Perplexity: 2636.141
Epoch: 7 Valid Perplexity: 2392.297
Epoch: 8 Learning rate: 0.062
Epoch: 8 Train Perplexity: 2257.246
Epoch: 8 Valid Perplexity: 1858.075
Epoch: 9 Lear